# Refined Power Analysis for RE

This notebook explores using the RPA attack for reverse-engineering.

 - You will first [explore scalar multipliers](#Scalar-multipliers) and the multiples they compute.
 - Then, you will study the [RPA attack](#RPA-attack).
 - Finally you will put the knowledge to use in [reverse-engineering](#Putting-it-together).

In [ ]:
import numpy as np

from pyecsca.ec.params import get_params, DomainParameters
from pyecsca.ec.context import DefaultContext, local
from pyecsca.ec.mod import Mod
from pyecsca.ec.point import Point
from pyecsca.ec.mult import *

## Scalar multipliers
Scalar multipliers come in all sorts of different shapes, sizes and colors. 🌈

In [ ]:
from pyecsca.ec.mult import LTRMultiplier, RTLMultiplier
from pyecsca.sca.re.rpa import MultipleContext

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

Compare the chain of multiples that is computed in the left-to-right vs right-to-left scalar multiplier for a given fixed scalar.
- What do these intermediate multiples (scalars) represent in the two cases?
- When are the chains the same? When are the sets of multiples the same?

**Bonus:** You can examine what the `always` and `complete` options to the `LTRMultipler` and `RTLMultipler` do and how they affect the multiples.

**Docs**<br/>
[LTRMultiplier](https://neuromancer.sk/pyecsca/api/pyecsca.ec.mult.binary.html#pyecsca.ec.mult.binary.LTRMultiplier)<br/>
[RTLMultiplier](https://neuromancer.sk/pyecsca/api/pyecsca.ec.mult.binary.html#pyecsca.ec.mult.binary.RTLMultiplier)<br/>
[MultipleContext](https://neuromancer.sk/pyecsca/api/pyecsca.sca.re.rpa.html#pyecsca.sca.re.rpa.MultipleContext)

In [ ]:
from itertools import zip_longest
params = get_params("secg", "secp256r1", "projective")
order = params.order
add = params.curve.coordinate_model.formulas["add-2007-bl"] # The formulas do not matter, but we need some
dbl = params.curve.coordinate_model.formulas["dbl-2007-bl"] # The formulas do not matter, but we need some

ltr_mult = LTRMultiplier(add, dbl, complete=False)
rtl_mult = RTLMultiplier(add, dbl, complete=False)

# Task: Fix a scalar
k = 123456789
P = params.generator

with local(MultipleContext()) as ltr_ctx:
    # Task: Init and multiply using the LTR multiplier here
    ltr_mult.init(params, params.generator)
    ltr_mult.multiply(k)

with local(MultipleContext()) as rtl_ctx:
    # Task: Init and multiply using the RTL multiplier here 
    rtl_mult.init(params, params.generator)
    rtl_mult.multiply(k)

# Task: Compare the ltr_ctx and rtl_ctx.
# You can look at their .points and .parents attributes.
ltr_multiples = set(ltr_ctx.points.values())
rtl_multiples = set(rtl_ctx.points.values())
print("LTR", sorted(ltr_multiples))
print("RTL", sorted(rtl_multiples))
print("scalar", k, bin(k)[2:])
print("LTR", " " * 26, "RTL")
for ltr_val, rtl_val in zip_longest(ltr_ctx.points.values(), rtl_ctx.points.values()):
    ltr_bin = f"{bin(ltr_val)[2:]:30}" if ltr_val is not None else " " * 30
    rtl_bin = f"{bin(rtl_val)[2:]:30}" if rtl_val is not None else " " * 30
    print(ltr_bin, rtl_bin)

## RPA attack
The RPA attack (by Louis Goubin, [A Refined Power-Analysis Attack on Elliptic Curve Cryptosystems](http://www.goubin.fr/papers/ecc-dpa.pdf)) works by introducing a zero-coordinate-point into the scalar multiplication conditionally on a secret key part and then detecting the presence of the zero via a side-channel. The target is static ECDH, in which case there is a static scalar $k$ and the target is willing to compute $[k]P$ for any (reasonable) point $P$.

### Smuggling the zero in

In the attack, the existence of zero-coordinate points is important.
They are either of the form $P_0 = (x, 0)$ or $P_0 = (0, y)$. The first form is less interesting, as it represents points of order 2, which are not present on prime-order elliptic curves often used in practice.

During the attack, the attacker can introduces these zero-coordinate points into the scalar multiplication conditionally on some secret key hypothesis. Upon input of $P = [r^{-1}]P_0$ into the scalar multiplier, the point $P_0$ will appear inside iff the multiplier computes the $r$-th multiple of the input point, as $[r]P = [r][r^{-1}]P_0 = P_0$. In the attack, the attacker uses this to confirm hypotheses about parts of the secret key.

### Detecting the zero

Now that we have a way of getting the zero in, we need a way of detecting it. We will use a power (or EM) side-channel and the fact that arithmetic operations with zero values will leak in a different way than operations with non-zero values. The diagram below shows the idea (which is very similar to DPA). We will collect two groups of traces, one with the special point $P=[r^{-1}]P_0$ and one with random points $P$ input into the target. We will then average the traces in the two groups and subtract the average traces to form a Difference-of-Means (DoM) trace. If this traces has a peak we were able to distinguish between the two groups, which implies that the zero did occur in the "special point $P$" group, which in turn implies that the scalar multiplier computed the $r$-th multiple of the input point.

![](../img/rpa.svg)

### Nuances

The RPA attack will not work in the presence of scalar randomization countermeasures, as these effectively randomize the sequence of multiples computed while computing the $[k]P$. However, the attack is resistant against coordinate and curve randomization, because the zero-coordinate points resist randomization.

In [ ]:
from pyecsca.sca.re.rpa import rpa_point_0y, rpa_point_x0
from pyecsca.sca.trace import Trace
from pyecsca.ec.formula import FormulaAction
from pyecsca.sca.attack.leakage_model import HammingWeight

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

Practice the elements of the RPA attack.
 - Create a point P, such that $P_0$ occurs in the scalar multiplication $[k]P$ using the LTR multiplier but not the RTL multiplier.
 - Verify your construction of the point.
 - Construct the RPA oracle "box" from the figure above, compare it to a simulated one.

**Docs**<br/>
[rpa_point_0y](https://neuromancer.sk/pyecsca/api/pyecsca.sca.re.rpa.html#pyecsca.sca.re.rpa.rpa_point_0y)<br/>
[Mod](https://neuromancer.sk/pyecsca/api/pyecsca.ec.mod.html#pyecsca.ec.mod.Mod)<br/>
[affine_multiply](https://neuromancer.sk/pyecsca/api/pyecsca.ec.curve.html#pyecsca.ec.curve.EllipticCurve.affine_multiply)<br/>
[MultipleContext](https://neuromancer.sk/pyecsca/api/pyecsca.sca.re.rpa.html#pyecsca.sca.re.rpa.MultipleContext)<br/>
[combine module (average, subtract, ...)](https://neuromancer.sk/pyecsca/api/pyecsca.sca.trace.combine.html)<br/>
[process module (normalize, ...)](https://neuromancer.sk/pyecsca/api/pyecsca.sca.trace.process.html)<br/>

In [ ]:
P0 = rpa_point_0y(params)
print(P0)
# Task: Create a point P, such that P0 occurs in the scalar multiplication of [k]P using the LTR multiplier
#       but not the RTL multiplier.
# Hint: You can use params.curve.affine_multiply.
only_ltr_multiples = ltr_multiples.difference(rtl_multiples)
only_rtl_multiples = rtl_multiples.difference(ltr_multiples)

# Pick a multiple that apppears only in the LTR:
r = list(only_ltr_multiples)[5]
rmod = Mod(r, params.order)
P = params.curve.affine_multiply(P0, int(rmod.inverse()))
print(P)

In [ ]:
# Task: Verify that your construction of the point is correct by simulating the multipliers using the MultipleContext
#       as above and verifying that the point is present when it should be.
# Note: You will need to transform the points to the projective coordinate system using their
#       .to_model(params.curve.coordinate_model, params.curve) method
with local(MultipleContext()) as lctx:
    ltr_mult.init(params, P.to_model(params.curve.coordinate_model, params.curve))
    ltr_mult.multiply(k)
print("LTR")
for pt, val in lctx.points.items():
    if pt.X == 0:
        print("Yes", pt, val)
        break
else:
    print("No")

with local(MultipleContext()) as rctx:
    rtl_mult.init(params, P.to_model(params.curve.coordinate_model, params.curve))
    rtl_mult.multiply(k)
print("RTL")
for pt, val in rctx.points.items():
    if pt.X == 0:
        print("yes", pt, val)
        break
else:
    print("No")

In [ ]:
from pyecsca.sca.trace.combine import average, subtract
from pyecsca.sca.trace.process import normalize
from scipy.signal import find_peaks

def simulate_trace(mult: ScalarMultiplier, point: Point, scalar: int, params: DomainParameters) -> Trace:
    with local(DefaultContext()) as ctx:
        mult.init(params, point)
        mult.multiply(scalar)

    lm = HammingWeight()
    trace = []

    def callback(action):
        if isinstance(action, FormulaAction):
            for intermediate in action.op_results:
                leak = lm(intermediate.value)
                trace.append(leak)

    ctx.actions.walk(callback)
    return Trace(np.array(trace))

def simulated_oracle(mult: ScalarMultiplier, affine_point: Point, scalar: int, params: DomainParameters) -> bool:
    point = affine_point.to_model(params.curve.coordinate_model, params.curve, randomized=True)
    with local(MultipleContext()) as ctx:
        mult.init(params, point)
        mult.multiply(scalar)
    points = set(ctx.parents.keys())
    parents = set(sum((ctx.parents[point] for point in points), []))
    zero = any(map(lambda pt: pt.X == 0 or pt.Y == 0, parents)) # Did zero happen in some input point during the scalarmult?
    return zero

# Task: Use the "simulate_trace" function to implement the "rpa_oracle" function
def rpa_oracle(mult: ScalarMultiplier, affine_point: Point, scalar: int, params: DomainParameters) -> bool:
    num_target = 100
    num_random = 100
    # simulate a group of traces (e.g. 100) using "point" (this is P = [r^{-1}]P_0 from the diagram)
    target_traces = [normalize(simulate_trace(mult, affine_point.to_model(params.curve.coordinate_model, params.curve, randomized=True), scalar, params)) for _ in range(num_target)]
    # simulate a group of traces (e.g. 100) using random points
    random_traces = [normalize(simulate_trace(mult, params.curve.affine_random().to_model(params.curve.coordinate_model, params.curve, randomized=True), scalar, params)) for _ in range(num_random)]

    # average, subtract, and decide on the result
    random_avg = average(*random_traces)
    target_avg = average(*target_traces)
    diff_trace = normalize(subtract(random_avg, target_avg))
    peaks, props = find_peaks(diff_trace.samples, height=4)
    return len(peaks) != 0

# Task: Compare your "rpa_oracle" function to the "simulated_oracle" function that is the ground truth
print("LTR")
print("Simulated", simulated_oracle(ltr_mult, P, k, params))
print("RPA", rpa_oracle(ltr_mult, P, k, params))
print("RTL")
print("Simulated", simulated_oracle(rtl_mult, P, k, params))
print("RPA", rpa_oracle(rtl_mult, P, k, params))

## Putting it together

You will now put your knowledge of scalar multipliers and the RPA attack together, to turn it into a reverse-engineering technique.

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

1. Pick a scalar.
2. Simulate each multiplier with said scalar, collect multiples computed during the scalar multiplications.
3. Use the collected multiples to to figure out a way to distinguish between the scalar multipliers.
   - Assume that you have an oracle that you can ask whether a particular multiple gets computed by the target while computing the scalar multiplication. (Because you have one, as just demonstrated in the RPA attack section).
   - Try to find an optimal way with regards to number of oracle queries. 

**Docs**

In [ ]:
neg = params.curve.coordinate_model.formulas["neg"]

# These will be the scalar multipliers that we will be distinguishing.
multipliers = [
    LTRMultiplier(add, dbl, None, False, AccumulationOrder.PeqPR, True, True),
    LTRMultiplier(add, dbl, None, True, AccumulationOrder.PeqPR, True, True),
    RTLMultiplier(add, dbl, None, False, AccumulationOrder.PeqPR, True),
    RTLMultiplier(add, dbl, None, True, AccumulationOrder.PeqPR, False),
    SimpleLadderMultiplier(add, dbl, None, True, True),
    BinaryNAFMultiplier(add, dbl, neg, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    BinaryNAFMultiplier(add, dbl, neg, None, ProcessingDirection.RTL, AccumulationOrder.PeqPR, True),
    WindowNAFMultiplier(add, dbl, neg, 3, None, AccumulationOrder.PeqPR, True, True),
    WindowNAFMultiplier(add, dbl, neg, 4, None, AccumulationOrder.PeqPR, True, True),
    WindowNAFMultiplier(add, dbl, neg, 5, None, AccumulationOrder.PeqPR, True, True),
    WindowBoothMultiplier(add, dbl, neg, 3, None, AccumulationOrder.PeqPR, True, True),
    WindowBoothMultiplier(add, dbl, neg, 4, None, AccumulationOrder.PeqPR, True, True),
    WindowBoothMultiplier(add, dbl, neg, 5, None, AccumulationOrder.PeqPR, True, True),
    SlidingWindowMultiplier(add, dbl, 3, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    SlidingWindowMultiplier(add, dbl, 4, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    SlidingWindowMultiplier(add, dbl, 5, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    SlidingWindowMultiplier(add, dbl, 3, None, ProcessingDirection.RTL, AccumulationOrder.PeqPR, True),
    SlidingWindowMultiplier(add, dbl, 4, None, ProcessingDirection.RTL, AccumulationOrder.PeqPR, True),
    SlidingWindowMultiplier(add, dbl, 5, None, ProcessingDirection.RTL, AccumulationOrder.PeqPR, True),
    FixedWindowLTRMultiplier(add, dbl, 3, None, AccumulationOrder.PeqPR, True),
    FixedWindowLTRMultiplier(add, dbl, 4, None, AccumulationOrder.PeqPR, True),
    FixedWindowLTRMultiplier(add, dbl, 5, None, AccumulationOrder.PeqPR, True),
    FixedWindowLTRMultiplier(add, dbl, 8, None, AccumulationOrder.PeqPR, True),
    FixedWindowLTRMultiplier(add, dbl, 16, None, AccumulationOrder.PeqPR, True),
    FullPrecompMultiplier(add, dbl, None, True, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True, True),
    FullPrecompMultiplier(add, dbl, None, False, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True, True),
    BGMWMultiplier(add, dbl, 2, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    BGMWMultiplier(add, dbl, 3, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    BGMWMultiplier(add, dbl, 4, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    BGMWMultiplier(add, dbl, 5, None, ProcessingDirection.LTR, AccumulationOrder.PeqPR, True),
    CombMultiplier(add, dbl, 2, None, AccumulationOrder.PeqPR, True),
    CombMultiplier(add, dbl, 3, None, AccumulationOrder.PeqPR, True),
    CombMultiplier(add, dbl, 4, None, AccumulationOrder.PeqPR, True),
    CombMultiplier(add, dbl, 5, None, AccumulationOrder.PeqPR, True)
]

In [ ]:
# Task: Pick a scalar and simulate each multiplier on it,
#       collect the multiples each multiplier does using the MultipleContext.
k = 123456789
mult_to_multiples = {}
for mult in multipliers:
    with local(MultipleContext()) as ctx:
        mult.init(params, params.generator)
        mult.multiply(k)
    mult_to_multiples[mult] = set(ctx.points.values())

To distinguish the multiplier we will consider all the multiples that get computed and try to find one that is computed by exactly half of the multipliers. Basically, we are doing binary search (which can be represented by a decition tree). This is not necessarily optimal, as we are making locally-optimal choices in a greedy manner. However, it is sufficient for our use case.


**Example**
![](../img/rpa_tree_example.png)

In [ ]:
# Task: Figure out a way to distinguish between the multipliers based on the multiples.
# Hint: You will end up building a tree.
from anytree import Node, RenderTree

def build_tree(mult_map, response=None) -> Node:
    all_multiples = set().union(*mult_map.values())
    if len(mult_map) == 1:
        return Node(set(mult_map.keys()), response=response)
    half = len(mult_map) // 2
    best_split = (half, None, None)
    for multiple in all_multiples:
        true_mults = set()
        false_mults = set()
        for mult, multiples in mult_map.items():
            if multiple in multiples:
                true_mults.add(mult)
            else:
                false_mults.add(mult)
        if (score := abs(len(true_mults) - half)) <= best_split[0]:
            best_split = (score, true_mults, false_mults)
            if score == 0:
                break
    if best_split[0] == half:
        return Node(set(mult_map.keys()), response=response)
    result = Node(multiple, response=response)
    
    true_mults = best_split[1]
    true_node = build_tree({mult: mult_map[mult] for mult in true_mults}, response=True)
    true_node.parent = result

    false_mults = best_split[2]
    false_node = build_tree({mult: mult_map[mult] for mult in false_mults}, response=False)
    false_node.parent = result

    return result

In [ ]:
tree = build_tree(mult_to_multiples)
print(RenderTree(tree).by_attr())